In [9]:
import sys
import config
from config import *
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import numpy as np

In [10]:
data = pd.read_csv('data_clusters.csv')

In [11]:
from config import *
client_credentials_manager = SpotifyClientCredentials(client_id=client_ID, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_ID,
                                                           client_secret=client_secret))

In [12]:
# Get input from the user
artist = input("Type in the artist: ")
title = input("Type in the song name: ")

Type in the artist:  rammstein
Type in the song name:  sonne


In [13]:
def search_song(title, artist, limit):
    
    try:
        results = sp.search(q=f"{title} {artist}", type='track', limit=limit)
        song_id = results['tracks']['items'][0]['id']
        return song_id
    except IndexError:
        print("Song not found!")
        return ""

In [14]:
song_id = search_song(title, artist, 5)
song_id

'4w3R0GZ2rScJMKlgPJzHRq'

In [18]:
audio_features = sp.audio_features([song_id])

In [19]:
audio_features
song = pd.DataFrame(audio_features)

In [20]:
features=song.select_dtypes(exclude=[object])

In [21]:
columns_to_drop = ['liveness', 'duration_ms', 'time_signature']
features.drop(columns=columns_to_drop, inplace=True)

In [23]:
import pickle

scaler_filename = "./feature_scaler.pickle"
with open(scaler_filename, "rb") as file:
    scaler = pickle.load(file)

user_song_audio_features = scaler.transform(features)
user_song_audio_features_df = pd.DataFrame(user_song_audio_features, columns = features.columns)

umap_filename = "./umap.pickle"
with open(umap_filename, "rb") as file:
    umap = pickle.load(file)

user_song_umap = umap.transform(user_song_audio_features_df)
user_song_umap_df = pd.DataFrame(user_song_umap, columns=["UMAP_1","UMAP_2"])

cluster_model_filename = "./kmeans.pickle"
with open(cluster_model_filename, "rb") as file:   
    model = pickle.load(file)
user_song_cluster = model.predict(user_song_umap_df)[0]

In [25]:
if song_id in data[data['hot_song']==1]['id'].tolist():
    result_1 = data[(data['hot_song']==1) & (data['cluster']== user_song_cluster)].sample(5)
    result_1 = pd.DataFrame(result_1)
    result_1['Spotify_Link'] = 'https://open.spotify.com/track/' + result_1['id']
    output = result_1[['Title', 'Artists', 'Spotify_Link']]
    print('Here are songs you may like:')
    display(output)
else:
    result_2 = data[(data['hot_song'] != 1) & (data['cluster']== user_song_cluster)].sample(5)
    result_2 = pd.DataFrame(result_2)
    result_2['Spotify_Link'] = 'https://open.spotify.com/track/' + result_2['id']
    output = result_2[['Title', 'Artists', 'Spotify_Link']]
    print('Here are songs you may like:')
    display(output)

     

Here are songs you may like:


,Title,Artists,Spotify_Link
1230,Days That End In Why,Morgan Wallen,https://open.spotify.com/track/4FhxN8PTyjkBVfw...
195,The A Team,Ed Sheeran,https://open.spotify.com/track/1VdZ0vKfR5jneCm...
282,Hallelujah,Panic! At The Disco,https://open.spotify.com/track/6TzJLISIitIzn1W...
1038,Everybody Business,Kehlani,https://open.spotify.com/track/6TrdeNEgbKuBqIT...
1407,Crossfaded,Don Toliver,https://open.spotify.com/track/2iu60eTIm2py6Ma...
